##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using TensorBoard in Notebooks

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

TensorBoard can be used directly within notebook experiences such as [Colab](https://colab.research.google.com/) and [Jupyter](https://jupyter.org/). This can be helpful for sharing results, integrating TensorBoard into existing workflows, and using TensorBoard without installing anything locally.

## Setup

Start by installing TF 2.0 and loading the TensorBoard notebook extension:

**For Jupyter users:** If you’ve installed Jupyter and TensorBoard into
the same virtualenv, then you should be good to go. If you’re using a
more complicated setup, like a global Jupyter installation and kernels
for different Conda/virtualenv environments, then you must ensure that
the `tensorboard` binary is on your `PATH` inside the Jupyter notebook
context. One way to do this is to modify the `kernel_spec` to prepend
the environment’s `bin` directory to `PATH`, [as described here][1].

[1]: https://github.com/ipython/ipykernel/issues/395#issuecomment-479787997


**For Docker users:** In case you are running a [Docker](https://docs.docker.com/install/) image of [Jupyter Notebook server using TensorFlow's nightly](https://www.tensorflow.org/install/docker#examples_using_cpu-only_images), it is necessary to expose not only the notebook's port, but the TensorBoard's port. Thus, run the container with the following command:

```
docker run -it -p 8888:8888 -p 6006:6006 \
tensorflow/tensorflow:nightly-py3-jupyter 
```

where the `-p 6006` is the default port of TensorBoard. This will allocate a port for you to run one TensorBoard instance. To have concurrent instances, it is necessary to allocate more ports. Also, pass `--bind_all` to `%tensorboard` to expose the port outside the container.

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

Import TensorFlow, datetime, and os:

In [2]:
import tensorflow as tf
import datetime, os

2022-02-21 22:34:08.237216: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-21 22:34:08.237242: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## TensorBoard in notebooks

Download the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and scale it:

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4431872/4422102 [==============================] - 1s 0us/step


Create a very simple model:

In [4]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

Train the model using Keras and the TensorBoard callback:

In [5]:
def train_model():
  
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()

2022-02-21 22:34:37.465013: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-21 22:34:37.465046: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-21 22:34:37.465067: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (adrian-Latitude-5491): /proc/driver/nvidia/version does not exist
2022-02-21 22:34:37.465327: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5


2022-02-21 22:34:37.659059: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 [==============================] - 6s 3ms/step - loss: 0.4930 - accuracy: 0.8226 - val_loss: 0.4290 - val_accuracy: 0.8443
Epoch 2/5
  41/1875 [..............................] - ETA: 4s - loss: 0.4155 - accuracy: 0.8537

2022-02-21 22:34:43.312174: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 96337920 exceeds 10% of free system memory.


...

1875/1875 [==============================] - 5s 3ms/step - loss: 0.3495 - accuracy: 0.8706 - val_loss: 0.3748 - val_accuracy: 0.8633
Epoch 4/5
  38/1875 [..............................] - ETA: 5s - loss: 0.3489 - accuracy: 0.8701

2022-02-21 22:34:53.245945: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 96337920 exceeds 10% of free system memory.


1875/1875 [==============================] - 5s 3ms/step - loss: 0.3276 - accuracy: 0.8785 - val_loss: 0.3493 - val_accuracy: 0.8693
Epoch 5/5
  40/1875 [..............................] - ETA: 4s - loss: 0.3164 - accuracy: 0.8750

2022-02-21 22:34:58.208229: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 96337920 exceeds 10% of free system memory.


1875/1875 [==============================] - 5s 3ms/step - loss: 0.3139 - accuracy: 0.8842 - val_loss: 0.3410 - val_accuracy: 0.8758


Start TensorBoard within the notebook using [magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html):

In [6]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard.png?raw=1"/> -->

You can now view dashboards such as scalars, graphs, histograms, and others. Some dashboards are not available yet in Colab (such as the profile plugin).

The `%tensorboard` magic has exactly the same format as the TensorBoard command line invocation, but with a `%`-sign in front of it.

You can also start TensorBoard before training to monitor it in progress:

In [7]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 22672), started 0:02:41 ago. (Use '!kill 22672' to kill it.)

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_two_runs.png?raw=1"/> -->

The same TensorBoard backend is reused by issuing the same command. If a different logs directory was chosen, a new instance of TensorBoard would be opened. Ports are managed automatically. 

Start training a new model and watch TensorBoard update automatically every 30 seconds or refresh it with the button on the top right:

In [8]:
train_model()

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5039 - accuracy: 0.8206 - val_loss: 0.4415 - val_accuracy: 0.8407
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3845 - accuracy: 0.8581 - val_loss: 0.3800 - val_accuracy: 0.8674
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3497 - accuracy: 0.8709 - val_loss: 0.3686 - val_accuracy: 0.8688
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3294 - accuracy: 0.8789 - val_loss: 0.3430 - val_accuracy: 0.8766
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3142 - accuracy: 0.8832 - val_loss: 0.3387 - val_accuracy: 0.8780


You can use the `tensorboard.notebook` APIs for a bit more control:

In [9]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs (started 0:03:33 ago; pid 22672)


In [10]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

Selecting TensorBoard with logdir logs (started 0:03:36 ago; port 6006, pid 22672).


<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_tall.png?raw=1"/> -->